## Prepare the latest sales record
6 months sales record history to reset the inventory aging time.
The guilty inventory count starts from the month of no sales.

1.  Delete the blank rows of the excel to have the header as 1st row.
2.  Save the excel by csv

In [16]:
import logging
import sqlite3

import pandas as pd

logging.basicConfig(
    level=logging.DEBUG, format=" %(asctime)s -  %(levelname)s -  %(message)s"
)

In [48]:
# Connect to sqlite3
conn = sqlite3.connect("guilty_inventory.sqlite3")
c = conn.cursor()

# Create tables
# t_sales table
c.execute(
    """
create table if not exists t_sales (sales_month text, sales_model text,
sales_qty int, sales_amt float, primary key(sales_month, sales_model))
"""
)

# t_purchase table
c.execute(
    """
create table if not exists t_purchase (purchase_month text, purchase_model text,
purchase_qty int, purchase_amt float, primary key(purchase_month, purchase_model))
"""
)

# t_inv table
c.execute(
    """
create table if not exists t_inv (inv_month text, inv_model text, inv_cat int,
inv_qty int, inv_amt float, inv_age int, primary key(inv_month, inv_model))
"""
)

# t_business_unit table
c.execute(
    """
create table if not exists t_business_unit (prod_category int primary key,
business_unit text)
"""
)

# t_rate table
c.execute(
    """
create table if not exists t_business_unit (rate_month text primary key,
rate_yen float)
"""
)

# t_guilty_base
c.execute(
    """
create table if not exists t_guilty_base (
inv_month text, inv_model text, inv_cat int, business_unit text,
inv_age int, rate_yen float, inv_qty int, inv_KMXN float,
inv_MYEN float, sales_n0_Q int, sales_n0_KMXN float,
sales_n0_MYEN float, sales_n1_Q int, sales_n1_KMXN float,
sales_n1_MYEN float, sales_n2_Q int, sales_n2_KMXN float,
sales_n2_MYEN float, sales_n3_Q int, sales_n3_KMXN float,
sales_n3_MYEN float, sales_n4_Q int, sales_n4_KMXN float,
sales_n4_MYEN float, sales_n5_Q int, sales_n5_KMXN float,
sales_n5_MYEN float, sales_n6_Q int, sales_n6_KMXN float,
sales_n6_MYEN float, purchase_n0_Q int, purchase_n1_Q int,
purchase_n2_Q int, purchase_n3_Q int, purchase_n4_Q int,
purchase_n5_Q int, purchase_n6_Q int, total_n0_Q int, total_n1_Q int,
primary key(inv_month, inv_model))
"""
)

# t_guilty_mod_age
c.execute(
    """
create table if not exists t_guilty_mod_age (
inv_month text, inv_model text, inv_cat int, business_unit text,
inv_age int, rate_yen float, inv_qty int, inv_KMXN float,
inv_MYEN float, sales_n0_Q int, sales_n0_KMXN float,
sales_n0_MYEN float, sales_n1_Q int, sales_n1_KMXN float,
sales_n1_MYEN float, sales_n2_Q int, sales_n2_KMXN float,
sales_n2_MYEN float, sales_n3_Q int, sales_n3_KMXN float,
sales_n3_MYEN float, sales_n4_Q int, sales_n4_KMXN float,
sales_n4_MYEN float, sales_n5_Q int, sales_n5_KMXN float,
sales_n5_MYEN float, sales_n6_Q int, sales_n6_KMXN float,
sales_n6_MYEN float, purchase_n0_Q int, purchase_n1_Q int,
purchase_n2_Q int, purchase_n3_Q int, purchase_n4_Q int,
purchase_n5_Q int, purchase_n6_Q int, total_n0_Q int, total_n1_Q int,
primary key(inv_month, inv_model))
"""
)

# t_guilty_list
c.execute(
    """
create table if not exists t_guilty_mod_age (
inv_month text, inv_model text, inv_model_digest text, bussines_unit text,
inv_age int, inv_qty int, inv_KMXN float, inv_MYEN float, sales_n0_Q int,
sales_n0_KMXN float, sales_n1_Q int, sales_n1_KMXN float, sales_n2_Q int,
sales_n2_KMXN float, sales_n3_Q int, sales_n3_KMXN float, sales_n4_Q int,
sales_n4_KMXN float, sales_n5_Q int, sales_n5_KMXN float, sales_n6_Q int,
sales_n6_KMXN float, primary key(inv_month, inv_model))
"""
)

# t_guilty_digest
c.execute(
    """
create table if not exists t_guilty_mod_age (
inv_month text, inv_model_digest text, bussines_unit text, inv_qty int,
inv_KMXN float, inv_n1_Q int, sales_n0_Q int, sales_n0_KMXN float,
sales_n1_Q int, sales_n1_KMXN float, sales_n2_Q int, sales_n2_KMXN float,
sales_n3_Q int, sales_n3_KMXN float, sales_n4_Q int, sales_n4_KMXN float,
sales_n5_Q int, sales_n5_KMXN float,sales_n6_Q int, sales_n6_KMXN float,
primary key(inv_month, inv_model_digest))
"""
)

# Close connection to sqlite3
c.close()
conn.close()

In [25]:
def t_sales_update_csv(targetFile):
    # Sales record preparation from csv and add to sqlite3 t_sales table
    # Copy the original sheet to a new sheet & paste value and save as csv.
    # Billing doc. date must be formatted to date (ex: 2020/8/2)
    # Sort by Billing doc. is ideal.

    useCols = ["Billing doc. date", "Material", "Net Qty", "Net $"]

    df = pd.read_csv(targetFile, usecols=useCols, parse_dates=[0])
    df = df.rename(
        columns={
            "Billing doc. date": "sales_month",
            "Material": "sales_model",
            "Net Qty": "sales_qty",
            "Net $": "sales_amt",
        }
    )
    
    # Modify the sales month to have 1st date of the month
    df["sales_month"] = df["sales_month"].apply(lambda x: x.replace(day=1))
    grouped = df.groupby(["sales_month", "sales_model"], as_index=False)
    # print(grouped.sum())

    # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add grouped.sum() into t_sales table
        grouped.sum().to_sql("t_sales", conn, if_exists="append", index=None)
    except Exception as e:
        print(e)
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_sales_update_csv("sales-201909-202008.csv")

datetime64[ns]
UNIQUE constraint failed: t_sales.sales_month, t_sales.sales_model


In [49]:
def t_inv_update_csv(targetFile):
    # Inventory aging preparation from csv and add to sqlite3 t_inv table
    # Copy the original sheet to a new sheet & paste value and save as csv.
    # Add columun "inv_month" with date format like 2019/9/1 (day must be 1st day)

    useCols = ["CatNo", "Material", "Fecha", "Suma de Total Qty", "Suma de Total $", "inv_month"]

    df = pd.read_csv(targetFile, usecols=useCols, parse_dates=[5])
    df = df.rename(
        columns={
            "CatNo": "inv_cat",
            "Material": "inv_model",
            "Fecha": "inv_age",
            "Suma de Total Qty": "inv_qty",
            "Suma de Total $": "inv_amt"
        }
    )
    
    
    df = df.reindex(columns=['inv_month', 'inv_model', 'inv_cat', 'inv_qty', 'inv_amt', 'inv_age'])
    
    # Replace the aging date
    agedict = {'30D': 30, '60D': 60, '90D': 90, '180D': 180, '270D': 270, '360D': 360, 'MAS': 999}
    df = df.replace(agedict)
  
    # Modify the inv_month to have 1st date of the month, just in case.
    df["inv_month"] = df["inv_month"].apply(lambda x: x.replace(day=1))
    
#     logging.debug('df status :\n' + str(df))
#     logging.debug('df is' + str(df['inv_month'].dtype))

    # Grouping by in_month and inv_model
    grouped = df.groupby(["inv_month", "inv_model"], as_index=False)
    logging.debug('grouped.sum() status:\n' + str(grouped.sum()))

     # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add grouped.sum() into t_sales table
        grouped.sum().to_sql("t_inv", conn, if_exists="append", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_inv_update_csv("inv201909-202008.csv")

 2020-09-21 01:47:26,891 -  DEBUG -  grouped.sum() status:
      inv_month           inv_model  inv_cat  inv_qty    inv_amt  inv_age
0    2019-09-01             AL100DR      639      124   93444.92      300
1    2019-09-01             AL100RD      213       10   76563.30       30
2    2019-09-01           AL100RD/U      213       35    3192.70      180
3    2019-09-01           AL100RT/U      426      435   52122.81      450
4    2019-09-01            AL100TDN      426      149  108706.84      270
...         ...                 ...      ...      ...        ...      ...
5011 2020-08-01   SOLTUERHEXSEG5/16      195       50       0.50      999
5012 2020-08-01      SOLVARILL001/4      195        4       0.04      999
5013 2020-08-01      SOLVARILL15MTS      195        1       0.01      360
5014 2020-08-01  SOLVARILLA5/16-1MO      195        7     339.15       90
5015 2020-08-01    SORIELBONDJUMPER      390        5    1392.88      150

[5016 rows x 6 columns]


Done with no error
